# Imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *

import os
from Utils import cacheOnDisk, Load
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
PNP530Cardio126=PNP530+Cardio126

pd.set_option('display.expand_frame_repr', False)


done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!
end of function!!!


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'25122018'

# shuffled X matrices:

In [ ]:
the function gen_shuffled_x_matrices was copied to PredictionWrapper2.py

In [ ]:
Xfile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126.dat'
shuf_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126_shuffling/'
n_shuf=20
shuf_size=120

gen_shuffled_x_matrices(Xfile,shuf_dir,n_shuf,shuf_size)


# First analysis - not generalized:

## generate 10 bootstrap file:

In [ ]:
Xfile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126.dat'
X=pd.read_pickle(Xfile)

shuf_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126_shuffling/'

print ('X shape is: ',X.shape)
for i in range(10):
    print i
    x_shuf=pd.DataFrame(index=X.index, data=X.sample(frac=1).values)
    x_shuf=x_shuf.iloc[:120,:]
    print ('x_shuf shape is: ',x_shuf.shape)
    x_shuf_name=shuf_dir+'shuf%s.dat' %i
    x_shuf.to_pickle(x_shuf_name)
    

## generate Y_bs files:

In [ ]:
Yname='Maximal Troponin'
Yfolder='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData'



Y_file='%s/%s.xlsx' %(Yfolder,Yname)
Y_file= Y_file.replace(' ','')
target=pd.read_excel(Y_file).set_index('BD')
#     print target.head()
# target=target[Yname]

# target=target.dropna()

In [ ]:
Y_bs_folder='%s/%s_bsFiles' %(Yfolder,Yname)
Y_bs_folder=Y_bs_folder.replace(' ','')
if not isdir(Y_bs_folder):
    makedirs(Y_bs_folder)

for i in range(10):
    target_b=target.sample(frac=0.95)
    target_b=target_b.sort_index()
    f1='%s/bs_%s.xlsx' %(Y_bs_folder,i)
    target_b.to_excel(f1)
    
    print target_b.shape
    print target_b.index.nunique()
#     print target_b.head()
    


In [ ]:
the function generate_bsYs was copied to predictionWrapper.py

## predict bs_y's with normal X (positive controls):

In [ ]:
#I ran the following code from the terminal:
# ran it manually for each bs - each time changed manually the bs number in the code...

!python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py  /net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/MaximalTroponin/bs9_XGBreg20LOO_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/ -path_to_X /net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126.dat -path_to_Y /net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/MaximalTroponin_bsFiles/bs_9.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -predictor_params '{"learning_rate":[0.1,0.05,0.01,0.005],"n_estimators": [200,600,1000,1400],"reg_alpha":[0,1,5,10],"gamma":[0,1,5,10],"subsample":[0.5,0.7],"max_depth":[1,3,5,7,9]}' -mem_def 1 -n_cols_per_job 1 –nPreSelectedFeatures 20 -k_folds 0

## predict y with shuffled X (neg controls):

In [ ]:
# the following code is running from the terminal:
# this is not efficient, couldnt find a way to parallelize the loop:
!python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionWrapper

## plot correlation between y_pred and y in original prediction:

In [ ]:
 the function adjusted_roundup was copied to myplots.py and the function plot_ypred_y_corr was copied to predictionWrapper2.py

In [ ]:
Yname='Maximal CPK'
Y_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/%s.xlsx' %Yname
pred_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/XGBreg20LOO\
_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/'
output_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/%s/' %Yname

plot_ypred_y_corr(Y_file,Yname,pred_dir,output_dir)

In [ ]:
pred_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/XGBreg20LOO_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/'
pred_dir.split('/')[-2]

## collect pearson_r and expVar from all predictions (21) and plot

In [ ]:
#real experiment:
real_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/XGBreg20LOO\
_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/results_df.pkl'
real=pd.read_pickle(real_file)
real.head()

In [ ]:
sum_df=pd.DataFrame()
row=0
# shuffled data-negative controls:
shuf_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126_shuffling/'
for i in range(10):
    shuf_res_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/MaximalTroponin/\
shuff%s_XGBreg20LOO_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/results_df.pkl' %i
    try:
        shuf_res=pd.read_pickle(shuf_res_file)
        sum_df.loc['shuf%s' %i,'pearson_r']=shuf_res.loc['Maximal Troponin','pearson_r']
        sum_df.loc['shuf%s' %i,'expVar']=shuf_res.loc['Maximal Troponin','explained_variance_score']
        sum_df.loc['shuf%s' %i,'color']=0  
        if i==0: sum_df.loc['shuf%s' %i,'label']=True
        else: sum_df.loc['shuf%s' %i,'label']=False
#         print shuf_res.head()
    except:
            print 'the file doesnt exist'
#positive controls:            
for j in range(10):
    bs_res_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/MaximalTroponin/\
bs%s_XGBreg20LOO_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/results_df.pkl' %j
    try:
        bs_res=pd.read_pickle(bs_res_file)
        sum_df.loc['bs%s' %j,'pearson_r']=bs_res.loc['Maximal Troponin','pearson_r']
        sum_df.loc['bs%s' %j,'expVar']=bs_res.loc['Maximal Troponin','explained_variance_score']
        sum_df.loc['bs%s' %j,'color']=1      
        if j==0: sum_df.loc['bs%s' %j,'label']=True
        else: sum_df.loc['bs%s' %j,'label']=False
#         print shuf_res.head()
    except:
            print 'the file doesnt exist'

#real experiment:
real_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/XGBreg20LOO\
_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/results_df.pkl'
real=pd.read_pickle(real_file)
sum_df.loc['real' ,'pearson_r']=real.loc['Maximal Troponin','pearson_r']
sum_df.loc['real' ,'expVar']=real.loc['Maximal Troponin','explained_variance_score']
sum_df.loc['real' ,'color']=2
sum_df.loc['real','label']=True

            
sum_df
    

In [ ]:
sum_df=sum_df.dropna(how='any')
fig2,ax2=plt.subplots(figsize=(7,7))
colorList=['red','green','navy']
labelList=['neg control (shuffled X)','pos control (bootstrapped y)','real data']
for i in range(sum_df.shape[0]):
    if sum_df.iloc[i,3]:
        ax2.scatter(x=sum_df.iloc[i,0],y=sum_df.iloc[i,1],color=colorList[int(sum_df.iloc[i,2])],label=labelList[int(sum_df.iloc[i,2])],s=100)
    else:
        ax2.scatter(x=sum_df.iloc[i,0],y=sum_df.iloc[i,1],color=colorList[int(sum_df.iloc[i,2])],s=100)
ax2.set_xlabel('pearson_r',fontsize='large')
ax2.set_ylabel('explained variance',fontsize='large')
ax2.set_title('Model performance - Maximal Troponin')
ax2.legend()
plt.show()

## predict noral y with predicted Age and Gender as X

In [ ]:
outputname='XGBreg20LOO_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar'
'_'.join(outputname.split('_')[:-2])+'_byPredictedAgeGender'

## find a way to generalize the process!!!

1. repeat for maximal_CPK, AST, ostial CX disease, D1 DISEASE
2. repeat for major classification results
3. repeat for major multiclass results? (optimize by kappa and not f1)

In [ ]:
Yname='Maximal CPK'
generate_bsYs(Yname)

In [ ]:
def gen_pos_neg_controls_plot(Yname,shuf_dir,n_permut,output_dir_Y):
    
    YnameNoSpace=Yname.replace(' ','')
    sum_df=pd.DataFrame()
    
    # shuffled data-negative controls:
    for i in range(n_permut):
        shuf_res_file=output_dir_Yshuf_dir+'results_df.pkl' %(YnameNoSpace,i)
        try:
            shuf_res=pd.read_pickle(shuf_res_file)
            sum_df.loc['shuf%s' %i,'pearson_r']=shuf_res.loc[Yname,'pearson_r']
            sum_df.loc['shuf%s' %i,'expVar']=shuf_res.loc[Yname,'explained_variance_score']
            sum_df.loc['shuf%s' %i,'color']=0  
            if i==0: sum_df.loc['shuf%s' %i,'label']=True
            else: sum_df.loc['shuf%s' %i,'label']=False
    #         print shuf_res.head()
        except:
                print 'the file  %s doesnt exist' %shuf_res_file
   
    #positive controls:            
    for j in range(n_permut):
        bs_res_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/%s/\
bs%s_XGBreg20LOO_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/results_df.pkl' %(YnameNoSpace,j)
        try:
            bs_res=pd.read_pickle(bs_res_file)
            sum_df.loc['bs%s' %j,'pearson_r']=bs_res.loc[Yname,'pearson_r']
            sum_df.loc['bs%s' %j,'expVar']=bs_res.loc[Yname,'explained_variance_score']
            sum_df.loc['bs%s' %j,'color']=1      
            if j==0: sum_df.loc['bs%s' %j,'label']=True
            else: sum_df.loc['bs%s' %j,'label']=False
    #         print shuf_res.head()
        except:
                print 'the file doesnt exist'

    #real experiment:
    real_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/XGBreg20LOO\
_randomSearch_25_byRepFeatPCA10percVDJ0999PredictedAgeGender_expVar/results_df.pkl'
    real=pd.read_pickle(real_file)
#     print real
    sum_df.loc['real' ,'pearson_r']=real.loc[Yname,'pearson_r']
    sum_df.loc['real' ,'expVar']=real.loc[Yname,'explained_variance_score']
    sum_df.loc['real' ,'color']=2
    sum_df.loc['real','label']=True
    
    
    #X=predictedAgeGenderOnly:
#     try:
    AgeGender_file='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/%s/\
XGBreg20LOO_randomSearch_25_byPredictedAgeGender_expVar/results_df.pkl' %YnameNoSpace
    AgeGender=pd.read_pickle(AgeGender_file)
    sum_df.loc['predAgeGender' ,'pearson_r']=AgeGender.loc[Yname,'pearson_r']
    sum_df.loc['predAgeGender' ,'expVar']=AgeGender.loc[Yname,'explained_variance_score']
    sum_df.loc['predAgeGender' ,'color']=3
    sum_df.loc['predAgeGender','label']=True
#     except:
#         print 'no predAgeGender file'
    print sum_df
    
    sum_df=sum_df.dropna(how='any')
    fig2,ax2=plt.subplots(figsize=(7,7))
    colorList=['red','green','navy','orange']
    labelList=['neg control (shuffled X)','pos control (bootstrapped y)','real data','onlyByPredAgeGender']
    for i in range(sum_df.shape[0]):
        if sum_df.iloc[i,3]:
            ax2.scatter(x=sum_df.iloc[i,0],y=sum_df.iloc[i,1],color=colorList[int(sum_df.iloc[i,2])],label=labelList[int(sum_df.iloc[i,2])],s=100)
        else:
            ax2.scatter(x=sum_df.iloc[i,0],y=sum_df.iloc[i,1],color=colorList[int(sum_df.iloc[i,2])],s=100)
    ax2.set_xlabel('pearson_r',fontsize='large')
    ax2.set_ylabel('explained variance',fontsize='large')
    ax2.set_title('Model performance - %s' %Yname)
    ax2.legend()
    plt.show()



In [ ]:
Yname='Maximal CPK'
shuf_dir='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/X_withPredictedAgeGender_Cardio126_shuffling/'
gen_pos_neg_controls_plot(Yname,shuf_dir)

## run for GRACE score

In [ ]:
Yname='GRACE Score'
generate_bsYs(Yname)

In [ ]:
# run prediction for 10 bootstrapped and 10 shuffled
# plot pearson r and exaplined variance for all and compare
# repeat for other targets

In [ ]:
Yname='GRACE Score'
gen_pos_neg_controls_plot(Yname)

In [ ]:
Xpath = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/PredictedAgeGender.dat' 
XpredAgeGender=pd.read_pickle(Xpath)
XpredAgeGender.shape

In [ ]:
XpredAgeGender.head()

## repeat for maximal_CPK, AST, ostial CX disease, D1 DISEASE

In [ ]:
XpredAgeGender_Cardio126=XpredAgeGender.loc[Cardio126,:]
XpredAgeGender_Cardio126.shape

In [ ]:
XCardio_path='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/PredictedAgeGender_Cardio126.dat'
XpredAgeGender_Cardio126.to_pickle(XCardio_path)

# automated run of all phenotypes using 20 permutations:
multi predictions didn't work

used the scripts i folder '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/pyFiles_runBinary_predictions/'

## get p-values from all runs:

In [ ]:
def get_p_values_from_all_runs(result_dir,dirNameList,pCut):
    
    allPredSumDF=pd.DataFrame()
    folder_list=listdir(result_dir)
    print ('number of folders withing directory is: ',len(folder_list))
    foldersToUse=[]
    for n,f in enumerate(folder_list):
#         print 'extracting shuffling results...'
#         if n%20==0: print n
        for d in dirNameList:
            if d in f:
                foldersToUse.append(f)
                if 'mapped' in f:
                    Yname=''.join([f.split('_')[0],f.split('_')[1]])
                else:
                    Yname=f.split('_')[0]

                for name in glob.glob(result_dir+f+'/sum_df_*'):       
                    sumDFfile=name
                    sumDF=pd.read_excel(sumDFfile)
                    shufList=[x for x in sumDF.index if ('shuf' in x) and ('mean' not in x)]
                    eval_metric1=sumDF.columns[0]
                    eval_metric2=sumDF.columns[1]
                    eval_metric_list=[eval_metric1, eval_metric2]
                    for em in eval_metric_list:

                        real_metric=sumDF.loc['real',em]
                        rowname=Yname+'_'+em
                        allPredSumDF.loc[rowname,'real']=real_metric
                        for shuf in shufList:
                            allPredSumDF.loc[rowname,shuf]=sumDF.loc[shuf,em]
    print ('number of predictions with result summary is: ',len(foldersToUse))
    print ('required p-value according to bonferroni: ',0.05/len(foldersToUse))
    
    f1=result_dir+'predSum_%s.xlsx' %cdate
    allPredSumDF.to_excel(f1)
    
    def calc_obs_p(x):
        real=x[0]
        shufs=x[1:]
        sigs=[x for x in shufs if x>=real]
        p_obs=(float(1+len(sigs)))/(1+len(shufs))

        return p_obs

    allPredSumDF['p_obs']=allPredSumDF.apply(lambda x:calc_obs_p(x),axis=1)
    print ('minimal p_obs: ',allPredSumDF['p_obs'].min())
    print ('maximal p_obs: ',allPredSumDF['p_obs'].max())
    
    sigResults=allPredSumDF[allPredSumDF['p_obs']<=pCut]['p_obs']
    print sigResults.sort_values()
    
    phenotypesToPredict=[x.split('_')[0] for x in sigResults.index.tolist()]
    
    phenotypesToPredict=list(set(phenotypesToPredict))
    
    return phenotypesToPredict

In [ ]:
result_dir='%s/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/' %MyPath
dirNameList=['XGB20LOO_19perms_cont','logReg50_19perms','LR_19perms_MULTI','XGB20_19perms_multi']
pCut=0.1

phenotypesToPredict_01=get_p_values_from_all_runs(result_dir,dirNameList,pCut)
print phenotypesToPredict_01
print len(phenotypesToPredict_01)

In [ ]:
pCut=0.05

phenotypesToPredict_005=get_p_values_from_all_runs(result_dir,dirNameList,pCut)
print phenotypesToPredict_005
print len(phenotypesToPredict_005)

In [ ]:
1.0/2000

In [ ]:
df=Load('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/predictions2/Cardio126_diseasePhenotypes/GlucoseDisorder_XGB20_19perms_multi/shuf10_XGBmulticlass20_gridSearch_byRepFeatPCA10percVDJ0999PredictedAgeGender_optByF1macro/results_df.pkl')
df

In [ ]:
df

In [ ]:
a=list(np.random.randint(0,2,1000000))
b=list(np.random.randint(0,2,1000000))



In [ ]:
print a
print b

In [ ]:
np.sum(np.abs(np.array(a)-np.array(b)))

In [ ]:
pdist(np.stack((a,b),axis=1).T,'cityblock')[0]

In [ ]:
%%timeit

np.sum(np.abs(np.array(a)-np.array(b)))

In [ ]:
%%timeit

pdist(np.stack((a,b),axis=1).T,'cityblock')[0]

In [13]:
l=np.random.randint(0,11,1000000)

In [17]:
df=pd.DataFrame(np.random.randint(0,100,size=(10,10)))
df

,0,1,2,3,4,5,6,7,8,9
0,91,99,93,90,71,48,49,5,69,0
1,1,22,63,15,42,52,56,54,36,76
2,13,29,13,22,96,68,65,76,50,3
3,63,26,98,18,90,45,92,16,68,41
4,13,89,46,25,43,52,90,14,62,52
5,3,37,93,26,81,3,65,41,73,71
6,83,18,96,85,11,26,75,66,99,23
7,64,71,89,88,59,25,1,88,98,36
8,19,53,59,58,57,43,36,67,42,67
9,92,1,34,15,26,73,40,51,90,68


In [18]:
df.at[1,3]

15

In [19]:
df.iat[1,3]

15

In [21]:
df.iat[1,2:4]

ValueError: iAt based indexing can only have integer indexers